In [14]:
import pandas as pandas
import json as json
import lightgbm as lgb
import warnings
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
from sklearn.metrics import classification_report
import joblib as joblib

### Load data


In [3]:
data_train_lgbm = pandas.read_csv('../data/salary.train.processed.csv').set_index('id')
data_test_lgbm= pandas.read_csv('../data/salary.test.processed.csv').set_index('id')
data_train_lgbm.head(5)

,age-group,education-num,native-country,capitalgain,capitalloss,hoursperweek,occupation_prof-specialty,occupation_tech-support,occupation_exec-managerial,occupation_machine-op-inspct,...,relationship_own-child,relationship_unmarried,race_white,race_amer-indian-eskimo,race_black,race_asian-pac-islander,race_other,sex_male,sex_female,label
id,,,,,,,,,,,,,,,,,,,,,
26890,3.0,16.0,5,0.0,0.0,3.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
933,3.0,14.0,5,4.0,0.0,3.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
25596,2.0,10.0,5,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12949,2.0,13.0,5,0.0,0.0,3.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6681,1.0,9.0,5,0.0,0.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Train model

- Train model with best parameter and using class-weight
```
'n_estimators': 350,
'max_depth': 13,
'min_samples_leaf': 2,
'min_samples_split': 6,
'criterion': 'entropy',
'max_features': 0.475336315954348
 ```

In [5]:
X_full = data_train_lgbm.drop(['label'], axis='columns')
y_full = data_train_lgbm['label']

- Calculate scale pos weight

In [7]:
scale_pos_weight = len(y_full[y_full == 0]) / len(y_full[y_full == 1])
print(f"Calculated scale_pos_weight: {scale_pos_weight:.4f}")

Calculated scale_pos_weight: 1.3882


- Train

In [21]:
best_lgbm_params = {
    'n_estimators': 350,
    'max_depth': 13,
    'min_child_samples': 2, 
    'feature_fraction': 0.475336315954348

}

In [22]:
warnings.filterwarnings('default')
lgbm_model_final = lgb.LGBMClassifier(
    **best_lgbm_params,   
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)
lgbm_model_final.fit(X_full, y_full)

[LightGBM] [Warning] feature_fraction is set=0.475336315954348, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.475336315954348
[LightGBM] [Warning] feature_fraction is set=0.475336315954348, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.475336315954348
[LightGBM] [Info] Number of positive: 7001, number of negative: 9719
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001024 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 127
[LightGBM] [Info] Number of data points in the train set: 16720, number of used features: 48
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.418720 -> initscore=-0.328030
[LightGBM] [Info] Start training from score -0.328030


LGBMClassifier(feature_fraction=0.475336315954348, max_depth=13,
               min_child_samples=2, n_estimators=350, n_jobs=-1,
               random_state=42, scale_pos_weight=1.3882302528210255)